In [26]:
import os

import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw

In [2]:
def tail(f, lines=1, _buffer=4098):
    """Tail a file and get X lines from the end"""
    # place holder for the lines found
    lines_found = []

    # block counter will be multiplied by buffer
    # to get the block size from the end
    block_counter = -1

    # loop until we find X lines
    while len(lines_found) < lines:
        try:
            f.seek(block_counter * _buffer, os.SEEK_END)
        except IOError:  # either file is too small, or too many lines requested
            f.seek(0)
            lines_found = f.readlines()
            break

        lines_found = f.readlines()

        # decrement the block counter to get the
        # next X bytes
        block_counter -= 1

    return lines_found[-lines:]

In [3]:
def get_bond_indices(filename):
    f = open(filename)
    lines = tail(f, lines=56)
    f.close()

    split_lines = [line.split() for line in lines[:-9]]

    for line in split_lines:
        del line[0:2]
        line[0] = line[0][:-1]

    n_atoms = len(split_lines)
    bond_idx = np.zeros((n_atoms, n_atoms))

    indices = np.triu_indices(47)

    bond_idx[indices] = [float(item) for sublist in split_lines for item in sublist]
    return bond_idx

In [5]:
bonds = get_bond_indices('test.JANPA')

In [18]:
for i in range(bonds.shape[0]):
    for j in range(bonds.shape[1]):
#         print(int(round(bonds[i, j])))
        bonds[i, j] = int(round(bonds[i, j]))

In [25]:
np.where(np.triu(bonds, 1) == 2.)

(array([11, 18]), array([12, 19]))

In [29]:
with open('atom_list.txt') as f:
    atoms = f.readlines()

In [32]:
atoms = [atom.strip() for atom in atoms]

In [ ]:
mol = Chem.RWMol()

for atom in atoms:
    mol.AddAtom(Chem.Atom(atom))

for i, j in zip(*np.where(cmat[:, :] == 1)):
    mol.AddBond(int(i), int(j), Chem.BondType.SINGLE)

for at in mol.GetAtoms():
    at.SetNoImplicit(True)